## Trabajo Fin de Grado
### Gestor de Quirófanos
### Implementación de Algoritmo Particle Swarm para optimización del orden de las intervenciones quirúrgicas

#### Autor: Jesús García Armario

In [47]:
# Imports necesarios
import numpy as np
import matplotlib.pyplot as plt
import random
import math
import pandas as pd
import sys
sys.path.append('../')
from Heuristicas.Utils import Quirofano, ActoQuirurgico
from Genético.utilsGenetico import cromosomaAleatorio, evaluar, validar, distance, cruce, mutacion

In [48]:
# Importamos el listado preprocesado de una especialidad para las pruebas
filename = '..\\..\\Datos\\Listados_Preprocesados\\ListadoInterv_Preprocesado_MAXILOFACIAL.xlsx'
df = pd.read_excel(filename)
# Seleccionamos una muestra aleatoria de 100 pacientes
df = df.sample(n=100, random_state=1)
# Nos quedamos sólo con el NHC, Ponderación y duración
df = df[['NHC', 'PONDERACIÓN', 'DURACIÓN']]
# Sumamos a la duración 25 minutos por paciente para tener en cuenta el tiempo de preparación
df['DURACIÓN'] = df['DURACIÓN'] + 25
ventana = 30
# Dividimos la duración entre la ventana y redondeamos hacia arriba
df['DURACIÓN'] = df['DURACIÓN'].apply(lambda x: math.ceil(x/ventana))
# Creamos un set de actos quirúrgicos
actos_pendientes = list()
i = 0
for elemento in df.itertuples():
    actos_pendientes.append(ActoQuirurgico(i, elemento[3], elemento[1], elemento[2]))
    i += 1

In [49]:
# Creamos una población de 100 partículas válidas
def crear_poblacion(tiempos, quirofanos, dias, actos_pendientes, n):
    poblacion = []
    for i in range(n):
        poblacion.append(cromosomaAleatorio(tiempos, quirofanos, dias, actos_pendientes))
    return poblacion

In [50]:
# Iniciamos las velocidades de las partículas
def iniciar_velocidades(poblacion):
    pop = []
    for part in poblacion:
        particula = {}
        particula['cromosoma'] = part
        particula['velocidad'] = np.random.randint(2, len(part)) * 0.5
        pop.append(particula)
    return pop

In [54]:

# Evaluamos a cada partícula
def evaluar_poblacion(poblacion, tiempos = 16, quirofanos = 3, dias = 5, actos_pendientes = actos_pendientes):
    for part in poblacion:
        part['fitness'] = evaluar(part['cromosoma'], tiempos, quirofanos, dias, actos_pendientes)
    return poblacion

# Inicializamos la mejor partícula
def mejor_particula(poblacion):
    mejor = poblacion[0]
    for part in poblacion:
        if part['fitness'] < mejor['fitness']:
            mejor = part
    return mejor

In [57]:
# Creamos la población inicial
poblacion = crear_poblacion(16, 3, 5, actos_pendientes, 100)
# Iniciamos las velocidades
poblacion = iniciar_velocidades(poblacion)
# Evaluamos la población
poblacion = evaluar_poblacion(poblacion)
# Inicializamos la mejor partícula
mejor = mejor_particula(poblacion)

# Inicializamos los parámetros
w = 0.9
c1 = 0.5
c2 = 0.3
# Inicializamos el mejor global
mejor_global = mejor

# Inicializamos el contador de iteraciones
iteraciones = 0
# Inicializamos el contador de iteraciones sin mejora
iteraciones_sin_mejora = 0
# Inicializamos el número máximo de iteraciones sin mejora
max_iteraciones_sin_mejora = 100
# Inicializamos el número máximo de iteraciones
max_iteraciones = 1000

# Inicializamos el vector de fitness
fitness = []
# Inicializamos el vector de fitness de la mejor partícula
fitness_mejor = []

# Iniciamos el bucle
while iteraciones < max_iteraciones and iteraciones_sin_mejora < max_iteraciones_sin_mejora:
    # Actualizamos la mejor partícula global
    if mejor['fitness'][0] < mejor_global['fitness'][0]:
        mejor_global = mejor
        iteraciones_sin_mejora = 0
    else:
        iteraciones_sin_mejora += 1
    # Actualizamos las velocidades y posiciones
    for part in poblacion:
        # Actualizamos la velocidad
        part['velocidad'] = w * part['velocidad'] + c1 * random.random() * (mejor['fitness'][0] - part['fitness'][0]) + c2 * random.random() * (mejor_global['fitness'][0] - part['fitness'[0]])
        # Comprobamos si empeora la solución
        if part['velocidad'] < 0:
            part['velocidad'] = 0
        # Actualizamos la posición
        part['fitness'] = part['fitness'] + part['velocidad']
    # Evaluamos la población
    poblacion = evaluar_poblacion(poblacion)
    # Actualizamos la mejor partícula
    mejor = mejor_particula(poblacion)
    # Actualizamos el contador de iteraciones
    iteraciones += 1
    # Guardamos el fitness de la mejor partícula
    fitness_mejor.append(mejor['fitness'])
    # Guardamos el fitness medio de la población
    fitness.append(np.mean([part['fitness'] for part in poblacion]))

# Mostramos el resultado
print('Mejor solución: ', mejor_global['cromosoma'])
print('Fitness: ', mejor_global['fitness'])
print('Iteraciones: ', iteraciones)


KeyError: 'f'